In [90]:
import numpy as np
import pandas as pd
from typing import List,Any,Dict,Tuple
from collections import defaultdict
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from numpy import ndarray,dtype

# Text Representations

La **Representación de Texto** (**Text Representation**) es el proceso de transformar el lenguaje humano, inherentemente no estructurado y simbólico, en una forma estructurada y numérica que pueda ser procesada por algoritmos computacionales. Esta transformación es fundamental en el proceso de entrenamiento de modelos de ML y técnicas de análisis estadístico que operan exclusivamente sobre datos numéricos.

**Técnicas Básicas de Representación de Texto**: Bag-of-Words (BoW), TF-IDF, One-Hot Encoding, Bag-of-N-grams(BoN), Label Encoding

In [91]:
texts = [
  "el gato juega",
  "el perro corre",
  "el gato y el perro son amigos"
]

## BoW

**Bag-of-Words** (**BoW**) es un modelo de representación de texto que se basa en la frecuencia con la que aparecen las palabras en un documento. Este modelo tiene como problema que no toma en cuenta el orden o contexto de las palabras.

### Implementación con Numpy

In [69]:
def get_bow_vector(text:str, vocab:dict[str,int], unk_handling:bool=False) -> ndarray[tuple[int], dtype[Any]]:
  # convertir vocabulario a formato estandarizado si es necesario
  if isinstance(vocab, list):
    vocab: dict[str,int] = {word:idx for idx, word in enumerate(vocab)}
  
  # tokenizar el texto
  tokens: list[str] = text.split()
  
  # inicializar vector one-hot
  bow_vector: np.ndarray = np.zeros((len(vocab),), dtype=int)
  
  for token in tokens:
    # manejo de palabras desconocidas
    if token not in vocab.keys():
      if unk_handling: continue # ignorar palabras desconocidas
      else: raise ValueError(f"Token '{token}' no encontrado en el vocabulario.")
    
    # actualizar vector one-hot
    vector: ndarray[tuple[int], dtype[Any]] = np.zeros((len(vocab),), dtype=int)
    idx: int = vocab[token]
    vector[idx] = 1
    bow_vector += vector
  
  return bow_vector

### Prueba y Comparación

In [70]:
vocab = get_vocab_from_corpus(texts)
print(f"Vocabulario construido: {vocab}")

for text in texts:
  print(f"Texto '{text}'")
  onehot = get_bow_vector(text, vocab)
  print(onehot)

Vocabulario construido: {'amigos': 0, 'corre': 1, 'el': 2, 'gato': 3, 'juega': 4, 'perro': 5, 'son': 6, 'y': 7}
Texto 'el gato juega'
[0 0 1 1 1 0 0 0]
Texto 'el perro corre'
[0 1 1 0 0 1 0 0]
Texto 'el gato y el perro son amigos'
[1 0 2 1 0 1 1 1]


In [71]:
# Implementación con Scikit-Learn
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)

print(f"Vocabulario: {vectorizer.get_feature_names_out()}")
print(X.toarray())

Vocabulario: ['amigos' 'corre' 'el' 'gato' 'juega' 'perro' 'son']
[[0 0 1 1 1 0 0]
 [0 1 1 0 0 1 0]
 [1 0 2 1 0 1 1]]


## One-Hot Encoding

**One-Hot Encoding** es una técnica de representación de datos que consiste en convertir cada categoría única en un vector binario. Cada posición del vector representa una categoría y el valor 1 indica la presencia de esa categoría, mientras que el valor 0 indica su ausencia.

**Representación Matemática**: Dado un vocabulario $V = \{ w_{1},w_{2},\dots,w_{n} \}$ con $n$ términos únicos, cada término $w_{i}$ se representa como un vector $\overrightarrow{v}_{i} \in \{ 0,1 \}^n$ donde:

$$\overrightarrow{v}_{i}[j] = \left\{  \begin{array}{l}
1 & \text{si } j = i \\
0 & \text{en otro caso}
\end{array}\right.$$

A continuación se muestra una implementación con Numpy para realizar One-Hot Encoding y se compara que los resultados sean iguales a los obtenidos por Scikit-Learn.

### Implementación de One-Hot Encoding con Numpy

In [72]:
def get_onehot_vector(text:str, vocab:List[str], unk_handling:bool=False) -> ndarray[tuple[int], dtype[Any]]:
  # convertir vocabulario a formato estandarizado si es necesario
  if isinstance(vocab, list):
    vocab: dict[str,int] = {word:idx for idx, word in enumerate(vocab)}
  
  # tokenizar el texto
  tokens: list[str] = text.split()
  
  # inicializar vector one-hot
  onehot_vector: np.ndarray = np.zeros((len(tokens), len(vocab)), dtype=int)
  
  for idx, token in enumerate(tokens):
    # manejo de palabras desconocidas
    if token not in vocab.keys():
      if unk_handling: continue # ignorar palabras desconocidas
      else: raise ValueError(f"Token '{token}' no encontrado en el vocabulario.")
    
    # actualizar vector one-hot
    onehot_vector[idx, vocab[token]] = 1
  
  return onehot_vector

def get_vocab_from_corpus(corpus:List[str]) -> Dict[str,int]:
  unique_words: set[str] = set()
  for text in corpus:
    tokens: List[str] = text.split()
    unique_words.update(tokens)
    
  # ordenar vocabulario para consistencia
  vocab: Dict[str,int] = {word:idx for idx, word in enumerate(sorted(list(unique_words)))}
  return vocab

### Implementación con Scikit-Learn

In [73]:
def onehot_sklearn(texts:List[str], mode='document'):
  match mode:
    case 'document':
      # reprensentación a nivel de documento (Bag-of-Words)
      vectorizer = CountVectorizer(binary=True)
      onehot_matrix = vectorizer.fit_transform(texts).toarray()
      vocab = vectorizer.vocabulary_
      return onehot_matrix, vocab
    case 'word':
      # representación a nivel de palabra (One-Hot Encoding)
      vocab = { }
      idx_flat = []
      
      for text in texts: 
        for word in text.split():
          vocab[word] = vocab.get(word, len(vocab))
          idx_flat.append(vocab[word])
      
      encoder = OneHotEncoder(categories=[range(len(vocab))], sparse_output=False)
      onehot_matrix = encoder.fit_transform(np.array(idx_flat).reshape(-1, 1))
      return onehot_matrix, vocab

### Comparación


In [74]:
print("===== One-Hot Word Level =====")
vocab = get_vocab_from_corpus(texts)
print(f"Vocabulario construido: {vocab}")

for text in texts:
  print(f"Texto '{text}'")
  onehot = get_onehot_vector(text, vocab)
  print(onehot)

===== One-Hot Word Level =====
Vocabulario construido: {'amigos': 0, 'corre': 1, 'el': 2, 'gato': 3, 'juega': 4, 'perro': 5, 'son': 6, 'y': 7}
Texto 'el gato juega'
[[0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0]]
Texto 'el perro corre'
[[0 0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0 0]]
Texto 'el gato y el perro son amigos'
[[0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0]
 [1 0 0 0 0 0 0 0]]


In [75]:
print("===== SKLearn One-Hot Document Level =====")
onehot_doc, vocab_sklearn = onehot_sklearn(texts, mode='document')
print(f"Vocabulario construido: {vocab_sklearn}")
print("Matriz One-Hot Document-Word")
print(onehot_doc)

===== SKLearn One-Hot Document Level =====
Vocabulario construido: {'el': 2, 'gato': 3, 'juega': 4, 'perro': 5, 'corre': 1, 'son': 6, 'amigos': 0}
Matriz One-Hot Document-Word
[[0 0 1 1 1 0 0]
 [0 1 1 0 0 1 0]
 [1 0 1 1 0 1 1]]


In [76]:
print("===== SKLearn One-Hot Word Level =====")
onehot_word, vocab_sklearn_word = onehot_sklearn(texts, mode='word')
print(f"Vocabulario construido: {vocab_sklearn_word}")
print("Matriz One-Hot Word-Level")
print(onehot_word)

===== SKLearn One-Hot Word Level =====
Vocabulario construido: {'el': 0, 'gato': 1, 'juega': 2, 'perro': 3, 'corre': 4, 'y': 5, 'son': 6, 'amigos': 7}
Matriz One-Hot Word-Level
[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]


## Label Encoding

In [94]:
vocab = " ".join(texts).split()
encoder = LabelEncoder()
encoded_words = encoder.fit_transform(vocab)

print(f"Palabras únicas: {encoder.classes_}")
print(f"Codificación numérica: {np.arange(len(encoder.classes_))}")
print(f"Ejemplo de conversión '{texts[0]}':")
words = texts[0].split()
encoded_words = encoder.transform(words)
print(f"Original: {words} -> Codificado: {encoded_words}")

Palabras únicas: ['amigos' 'corre' 'el' 'gato' 'juega' 'perro' 'son' 'y']
Codificación numérica: [0 1 2 3 4 5 6 7]
Ejemplo de conversión 'el gato juega':
Original: ['el', 'gato', 'juega'] -> Codificado: [2 3 4]


## TF-IDF

In [92]:
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(texts)

print(f"Vocabulario: {tfidf_vectorizer.get_feature_names_out()}")
print(np.round(X_tfidf.toarray(), 2))

Vocabulario: ['amigos' 'corre' 'el' 'gato' 'juega' 'perro' 'son']
[[0.   0.   0.43 0.55 0.72 0.   0.  ]
 [0.   0.72 0.43 0.   0.   0.55 0.  ]
 [0.47 0.   0.55 0.36 0.   0.36 0.47]]


## N-Grams


In [77]:
texts = [
  "No me gusta este producto",
  "Me gusta mucho este producto"
]

In [78]:
# Configuración: Uni-gramas, Bi-gramas y Tri-gramas
vectorizer = CountVectorizer(ngram_range=(1,3))

X = vectorizer.fit_transform(texts)
feature_names = vectorizer.get_feature_names_out()
df = pd.DataFrame(X.toarray(), columns=feature_names, index=["Frase Negativa","Frase Positiva"])

In [79]:
# Se traspone para mejor lectura
df.T

,Frase Negativa,Frase Positiva
este,1,1
este producto,1,1
gusta,1,1
gusta este,1,0
gusta este producto,1,0
gusta mucho,0,1
gusta mucho este,0,1
me,1,1
me gusta,1,1
me gusta este,1,0
